In [85]:
import pandas as pd, numpy as np
import crime as cr
cr.load('county_population', full=True).head(3)

,county,fipscode,year,age,malepopulation,femalepopulation,totalpopulation
0,Adams,1.0,1990.0,0.0,2354.0,2404.0,4758.0
1,Adams,1.0,1990.0,1.0,2345.0,2375.0,4720.0
2,Adams,1.0,1990.0,2.0,2413.0,2219.0,4632.0


> The reason this dataset has nearly 400,000 rows is because<br>it gives stats on EACH INDIVIDUAL AGE, 0-90, so we have to<br>aggregate it ourselves. (number of rows = counties * years * 90).<br>It also has a 60 year time range (1990-2050)

In [82]:
df = cr.df('county_population')
df['age_range'] = "Adult"
df.loc[df.age <= 18, 'age_range'] = 'Minor'
df.insert(2, 'age_range', df.pop('age_range'))

df = df.drop(
        columns=['age', 'fipscode']
    ).rename(columns={
        'malepopulation': 'male',
        'femalepopulation': 'female',
        'totalpopulation': 'total',
    }).groupby([
        'county', 'year', 'age_range',
    ]).sum().reset_index()

dfm = df
dfm

,county,year,age_range,male,female,total
0,Adams,1990.0,Adult,90383.0,94282.0,184665.0
1,Adams,1990.0,Minor,41519.0,39525.0,81044.0
2,Adams,1991.0,Adult,93526.0,96801.0,190327.0
3,Adams,1991.0,Minor,42667.0,40626.0,83293.0
4,Adams,1992.0,Adult,96551.0,99314.0,195865.0
...,...,...,...,...,...,...
7803,Yuma,2048.0,Minor,1459.0,1401.0,2863.0
7804,Yuma,2049.0,Adult,3476.0,3586.0,7052.0
7805,Yuma,2049.0,Minor,1454.0,1391.0,2848.0
7806,Yuma,2050.0,Adult,3462.0,3572.0,7031.0


In [87]:
df = dfm.copy()

df = df.pivot(index=['county', 'year'], columns='age_range', values=['male', 'female', 'total']).reset_index()
df.columns = df.columns.droplevel()
df.columns.name = None
df.columns = ['county', 'year', 'adult_male', 'minor_male', 'adult_female', 'minor_female', 'adult_total', 'minor_total']
df = df[['county', 'year', 'minor_male', 'minor_female', 'adult_male', 'adult_female', 'minor_total', 'adult_total']]
df['total'] = df.minor_total + df.adult_total
df

,county,year,minor_male,minor_female,adult_male,adult_female,minor_total,adult_total,total
0,Adams,1990.0,41519.0,39525.0,90383.0,94282.0,81044.0,184665.0,265709.0
1,Adams,1991.0,42667.0,40626.0,93526.0,96801.0,83293.0,190327.0,273620.0
2,Adams,1992.0,43805.0,41716.0,96551.0,99314.0,85521.0,195865.0,281386.0
3,Adams,1993.0,45192.0,43042.0,100159.0,102389.0,88234.0,202548.0,290782.0
4,Adams,1994.0,46672.0,44453.0,104001.0,105667.0,91125.0,209668.0,300793.0
...,...,...,...,...,...,...,...,...,...
3899,Yuma,2046.0,1479.0,1415.0,3503.0,3605.0,2893.0,7105.0,9998.0
3900,Yuma,2047.0,1469.0,1411.0,3492.0,3598.0,2879.0,7092.0,9971.0
3901,Yuma,2048.0,1459.0,1401.0,3483.0,3585.0,2863.0,7073.0,9936.0
3902,Yuma,2049.0,1454.0,1391.0,3476.0,3586.0,2848.0,7052.0,9900.0


In [88]:
df.to_csv('county_stats/county_population.csv')